# The demo and test note of MNN

In [1]:
from mnn_pytorch import *
import numpy as np
import mnnbox as mnn
import torch
import time
torch.set_default_tensor_type(torch.DoubleTensor)

Init the key parameters for running

In [2]:
TOTAL_SAMPLES = 100000
EPOCHS = 10
BATCH_SIZE = 100000
NEURONS = 10
HIDDEN = 1

input_mean = np.random.uniform(1.9, 2.2, size=(TOTAL_SAMPLES, NEURONS))
input_std = np.random.uniform(9.8, 10.2, size=(TOTAL_SAMPLES, NEURONS))
target_mean = np.random.uniform(1.9, 2.2, size=(TOTAL_SAMPLES, NEURONS))
target_std = np.random.uniform(9.8, 10.2, size=(TOTAL_SAMPLES, NEURONS))

In [3]:
mnn_x = mnn.Input(value = np.empty((TOTAL_SAMPLES, NEURONS, 2)), name="input")
mnn_target = mnn.Variable(value=np.empty((TOTAL_SAMPLES, NEURONS, 2)), name="target")
mnn_all_weight = mnn.Variable(value=mnn.truncated_normal([NEURONS, NEURONS], stddev=0.1), name="weight")

mnn_x.value = np.stack([input_mean, input_std], axis=-1)
mnn_target.value = np.stack([target_mean, target_std], axis=-1)
mnn_activate = mnn.Activate(mnn_x)
cost = mnn.MSE(mnn_activate,mnn_target)

### Implement a simplest Mnn: Input -> Activate -> Out -> Loss

In [4]:
mnn_x.forward()
start_time1 = time.time()
mnn_activate.forward()
mnn_forward_time = time.time() - start_time1
print("The activation function of mnn_box cost:", mnn_forward_time)
cost.forward()
cost.backward()
start_time2 = time.time()
mnn_activate.backward()
mnn_backward_time = time.time() - start_time2
print("The activation function of mnn_box cost:", mnn_backward_time)
mnn_x.backward()
mnn_total = time.time() - start_time1
print("The time cost of one pass is:", mnn_total)

The activation function of mnn_box cost: 50.70154809951782
The activation function of mnn_box cost: 0.8059999942779541
The time cost of one pass is: 51.58254790306091


### Implement the pytorch version to compare the accuracy and efficiency

In [5]:
py_mean_in = torch.from_numpy(input_mean)
py_std_in = torch.from_numpy(input_std)
py_mean_target = torch.from_numpy(target_mean)
py_std_target = torch.from_numpy(target_std)

py_mean_in.requires_grad = True
py_std_in.requires_grad = True

In [6]:
print(py_mean_in.grad)
print(py_std_in.grad)

None
None


In [7]:
start_time3 = time.time()
activated_mean = Mnn_Activate_Mean.apply(py_mean_in, py_std_in)
activated_std = Mnn_Activate_Std.apply(py_mean_in, py_std_in)
py_forward_time = time.time() - start_time3
print("The forward activation of pytorch version cost:", py_forward_time)
py_mean_in.retain_grad()
py_std_in.retain_grad()
loss = loss_function(activated_mean, activated_std, py_mean_target, py_std_target)
loss.backward()
py_total = time.time() - start_time3
py_backward_time = py_total - py_forward_time
print("The backward activation of pytorch version cost:", py_backward_time)
print("Total time cost of one pass: ", py_total)

The forward activation of pytorch version cost: 2.209994316101074
The backward activation of pytorch version cost: 2.366999626159668
Total time cost of one pass:  4.576993942260742


# Summary

In [8]:
comp_loss = loss.clone().detach().numpy() - cost.value
py_mean_out = activated_mean.clone().detach().numpy().flatten()
py_std_out = activated_std.clone().detach().numpy().flatten()
py_mean_grad = py_mean_in.grad.clone().detach().numpy().flatten()
py_std_grad = py_std_in.grad.clone().detach().numpy().flatten()

In [36]:
mnn_mean_out = mnn_activate.value[:,:,0].flatten()
mnn_std_out = mnn_activate.value[:,:,1].flatten()
mnn_mean_grad = list(mnn_activate.gradients.values())[0][:,:,0].flatten()
mnn_std_grad = list(mnn_activate.gradients.values())[0][:,:,1].flatten()

In [38]:
comp_mean = np.abs(py_mean_out - mnn_mean_out)
comp_std = np.abs(py_std_out - mnn_std_out)
comp_mean_percent = comp_mean / np.abs(mnn_mean_out) * 100
comp_std_percent = comp_std / np.abs(mnn_mean_out) * 100
                   
comp_mean_grad = np.abs(py_mean_grad - mnn_mean_grad)
comp_std_grad = np.abs(py_std_grad - mnn_std_grad)
comp_mean_grad_percent = comp_mean_grad / np.abs(mnn_mean_grad) * 100
comp_std_grad_percent = comp_std_grad / np.abs(mnn_std_grad) * 100

In [42]:
print(comp_loss)
print(np.max(comp_mean), np.mean(comp_mean), np.max(comp_mean_percent), np.mean(comp_mean_percent))
print(np.max(comp_std), np.mean(comp_std), np.max(comp_std_percent), np.mean(comp_std_percent))
print(np.max(comp_mean_grad), np.mean(comp_mean_grad), np.max(comp_mean_grad_percent), np.mean(comp_mean_grad_percent))
print(np.max(comp_std_grad), np.mean(comp_std_grad), np.max(comp_std_grad_percent), np.mean(comp_std_grad_percent))

-0.0006371014258519381
4.361351624543319e-06 1.6230370322267925e-06 0.0034923480239515523 0.0012966999754939483
9.290188340738137e-05 3.71452568011119e-05 0.0743076041706477 0.029782921598036596
1.946102609138009e-09 1.5494510520514364e-09 0.46847957987365296 0.378407598558566
3.7259284681549974e-10 2.688457541055225e-10 0.2783340881207034 0.2163090677224089


### Pytorch版 与 原 Mnn 性能比较
#### 初始参数
* 测试模型： input -> activation -> output -> MSE Loss
* Total samples: 100000
* Neurons : 10
* Mean: [1.9, 2.2] 均匀分布 
* std : [9.8, 10.2] 均匀分布

即进行了2\*10\*100000 次激活计算
#### 运行时间
##### 原MNN
* forward： 50.70s
* backward:  0.80s
* total : 51.58s

##### Pytorch版
* forward: 2.21s
* backward: 2.37s
* total: 4.57s

pytorch版速度有极大提升，但由于pytorch框架限制，backward不方便利用之前的计算结果，故需要重新计算拖累运行效率，后续再考虑优化问题。

#### 计算误差比较：
* MSE loss ： 相差0.0006
* Mean Output: 最大误差 0.0035\%，平均 0.0013\%
* Std Output: 最大误差 0.074\%，平均 0.03\%
* Mean Backward gradient: 最大误差 0.47\%，平均0.38\%
* Std Backward gradient: 最大误差 0.28\%， 平均0.21\%